In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

In [2]:
### X = ["gt1", "gt2", "deficit", "ghidrs", "ea", "evert"]

In [3]:
class Estagio:

  def __init__(self, armazenamentos, afluencias, carga, custo_otimo):

    self.armazenamentos = armazenamentos
    self.afluencias = afluencias
    self.carga = carga

    self.results = {}
    self.custo_otimo = custo_otimo
  
  def calculate_alpha(self, polym, EA, EC, ghidrs, evert):

    x = EA + EC - ghidrs - evert

    return polym[0] * pow(x,4) + polym[1] * pow(x, 3) + polym[2] * pow(x, 2) + polym[3] * x + polym[4]  

  def interpolate_function(self, N_estagio):

      if N_estagio == 3:
      
        list_custo_otimo = [0, 0, 0]
      
      else:
        list_custo_otimo = []
        for key, value in self.custo_otimo.items():
          list_custo_otimo.append(value)

      z = np.polyfit(self.armazenamentos, list_custo_otimo, 4)
      
      return z

  def format_results(self, results):

    x = results['x']
    cost = results['fun']

    variables = {}
    
    variables["gt1"] = round(x[0], 4)
    variables["gt2"] = round(x[1], 4)
    variables["deficit"] = round(x[2], 4)
    variables["ghidrs"] = round(x[3], 4)
    variables["ea"] = round(x[4], 4)
    variables["evert"] = round(x[5], 4)
    variables["custo-imediato"] = round(cost, 4)

    return variables

  def minimize(self, N_estagio):

    polym = self.interpolate_function(N_estagio)
    
    for EA in self.armazenamentos:
      for EC in self.afluencias:
        
        function = lambda x: 10 * x[0] + 25 * x[1] + 500 * x[2] + 0.01 * x[5] + self.calculate_alpha(polym, EA, EC, x[3], x[5])

        constraints = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] + x[2] + x[3] - self.carga},
        {'type': 'eq', 'fun': lambda x: x[3] + x[4] + x[5] - EA - EC},
        {'type': 'ineq', 'fun': lambda x: x[0]},
        {'type': 'ineq', 'fun': lambda x: -x[0] + 15},
        {'type': 'ineq', 'fun': lambda x: x[1]},
        {'type': 'ineq', 'fun': lambda x: -x[1] + 10},
        {'type': 'ineq', 'fun': lambda x: x[2]},
        {'type': 'ineq', 'fun': lambda x: -x[2] + 65},
        {'type': 'ineq', 'fun': lambda x: x[3]},
        {'type': 'ineq', 'fun': lambda x: -x[3] + 76},
        {'type': 'ineq', 'fun': lambda x: x[4] - 100},
        {'type': 'ineq', 'fun': lambda x: -x[4] + 160},
        {'type': 'ineq', 'fun': lambda x: x[5]})

        res = minimize(function, (1, 1, 1, 1, 1, 1), method='SLSQP', constraints=constraints)

        self.results["EA"+str(EA)+"-EC"+str(EC)] = self.format_results(res)

    return self.results

  def calculate_custo_otimo(self):

    custo_otimo = {}
    for ea in self.armazenamentos:

      custo_ot = 0.0
      for ec in self.afluencias:

        custo_ot += (1/2) * self.results['EA'+str(ea)+'-EC'+str(ec)]['custo-imediato']

      custo_otimo[ea] = custo_ot

    return custo_otimo  


In [4]:
dict_vars = {3 : {"armazenamentos" : [160, 130, 100], "afluencias" : [42, 32], "carga":65},
             2:  {"armazenamentos" : [160, 130, 100], "afluencias" : [30, 18], "carga":65},
             1:  {"armazenamentos" : [160, 130, 100], "afluencias" : [26, 16], "carga":65}}

estagios = {}
for N_estagio, cenario in dict_vars.items():

  if N_estagio == 3:
    estagios[N_estagio] = Estagio(cenario['armazenamentos'], cenario['afluencias'], cenario['carga'], 0)

  else:
    estagios[N_estagio] = Estagio(cenario['armazenamentos'], cenario['afluencias'], cenario['carga'], estagios["custo_otimo-"+str(N_estagio + 1)])

  estagios["results-"+str(N_estagio)] = estagios[N_estagio].minimize(N_estagio)

  estagios["custo_otimo-"+str(N_estagio)] = estagios[N_estagio].calculate_custo_otimo()

  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: RankWarning: Polyfit may be poorly conditioned
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: RankWarning: Polyfit may be poorly conditioned
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: RankWarning: Polyfit may be poorly conditioned


In [5]:
table_3 = pd.DataFrame(data=estagios["results-3"])
table_2 = pd.DataFrame(data=estagios["results-2"])
table_1 = pd.DataFrame(data=estagios["results-1"])

In [6]:
table_3

,EA160-EC42,EA160-EC32,EA130-EC42,EA130-EC32,EA100-EC42,EA100-EC32
gt1,0.0,0.0,-0.0,3.0,15.0,15.0
gt2,0.0,0.0,-0.0,0.0,8.0,10.0
deficit,-0.0,-0.0,0.0,-0.0,-0.0,8.0
ghidrs,65.0,65.0,65.0,62.0,42.0,32.0
ea,137.0,127.0,107.0,100.0,100.0,100.0
evert,-0.0,-0.0,0.0,-0.0,-0.0,-0.0
custo-imediato,-0.0,-0.0,0.0,30.0,350.0,4400.0


In [7]:
table_2

,EA160-EC30,EA160-EC18,EA130-EC30,EA130-EC18,EA100-EC30,EA100-EC18
gt1,15.00,15.0000,15.0000,15.0000,15.0,15.0
gt2,0.00,10.0000,10.0000,10.0000,10.0,10.0
deficit,-0.00,-0.0000,-0.0000,-0.0000,10.0,22.0
ghidrs,50.00,40.0000,40.0000,40.0000,30.0,18.0
ea,140.00,138.0000,120.0000,108.0000,100.0,100.0
evert,-0.00,-0.0000,-0.0000,-0.0000,-0.0,0.0
custo-imediato,-187.09,109.6729,1024.0019,2008.7039,7775.0,13775.0


In [8]:
table_1

,EA160-EC26,EA160-EC16,EA130-EC26,EA130-EC16,EA100-EC26,EA100-EC16
gt1,15.0000,15.0000,15.0000,15.0000,15.0,15.0
gt2,10.0000,10.0000,10.0000,10.0000,10.0,10.0
deficit,0.0000,-0.0000,-0.0000,-0.0000,14.0,24.0
ghidrs,40.0000,40.0000,40.0000,40.0000,26.0,16.0
ea,146.0000,136.0000,116.0000,106.0000,100.0,100.0
evert,0.0000,-0.0000,-0.0000,-0.0000,0.0,-0.0
custo-imediato,-259.7627,797.9165,5626.1608,8975.2014,18175.0,23175.0
